In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import scipy.io as sc
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental import preprocessing
import time
from IPython.display import clear_output
import math 
from tensorflow import keras

In [4]:
def create_gen_input(batch_size, noise_size, n_class, seed=None):
        # create noise input
        noise = tf.random.normal([batch_size, noise_size], seed=seed)
        # Create categorical latent code
        label = tf.random.uniform([batch_size], minval=0, maxval=10, dtype=tf.int32, seed=seed)
        label = tf.one_hot(label, depth=n_class)
        # Create one continuous latent code
        c_1 = tf.random.uniform([batch_size, 1], minval=-1, maxval=1, seed=seed)
        return label, c_1, noise

In [40]:
label, c_1, noise = create_gen_input(256, 40, 60)
label[:1]
c_1.shape

TensorShape([256, 1])

In [61]:
def make_generator(blockSize):
    model = tf.keras.Sequential()
    model.add(layers.Dense(100, input_shape=(blockSize * 2+1,)))
    model.add(layers.Reshape([blockSize, 2, 1]))
    model.add(layers.Conv2D(16, (1, 2), strides=(1,2), activation="linear"))
    model.add(layers.Conv2D(32, (1,16), activation="linear", padding='same'))
    model.add(layers.Conv2D(16, (1,32), activation="linear", padding='same'))
    model.add(layers.Reshape((blockSize, 16, 1)))
    model.add(layers.AveragePooling2D((1,8)))
    model.add(layers.Dense(1))
    return model

In [57]:
def concat_inputs(input):
    concat_input = keras.layers.Concatenate()(input)
    return concat_input

In [62]:
test = make_generator(50)

In [60]:
test(inputs)

<tf.Tensor: shape=(256, 50, 2, 1), dtype=float32, numpy=
array([[[[ 2.0730140e-05],
         [-9.4024101e-03]],

        [[-1.4805847e-03],
         [-7.5080702e-03]],

        [[-8.6465984e-04],
         [ 4.3939147e-03]],

        ...,

        [[-3.8377027e-04],
         [-3.5374505e-05]],

        [[ 1.4174698e-04],
         [ 7.0992922e-03]],

        [[ 1.4865219e-03],
         [-2.4767770e-04]]],


       [[[ 1.6762196e-03],
         [-4.3025129e-03]],

        [[ 9.4503222e-04],
         [ 4.2397520e-03]],

        [[ 6.2645518e-04],
         [-1.5691620e-03]],

        ...,

        [[ 4.2374790e-04],
         [ 5.6771464e-03]],

        [[ 8.5038948e-04],
         [-6.0399198e-03]],

        [[ 1.9152770e-03],
         [ 3.5106887e-03]]],


       [[[-9.3946198e-04],
         [ 1.7283294e-04]],

        [[ 1.6924973e-03],
         [-2.1611548e-03]],

        [[ 5.4861675e-04],
         [-5.9129100e-04]],

        ...,

        [[-5.4390705e-04],
         [ 8.1374298e-04]],

 

In [26]:
def symbol_classifier(blocksize):
    model = keras.Sequential()
    model.add(layers.Conv2D(16, (1, 2), padding='same', activation='relu', input_shape=(blocksize, 2, 1)))
    model.add(layers.MaxPooling2D((1, 1)))
    model.add(layers.Conv2D(32, (4,1), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((1, 1)))
    model.add(layers.Conv2D(64, (4,1), strides=(1,2),padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((1, 1)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(20))
    return model

In [50]:
test = symbol_classifier(50)
test.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 50, 2, 16)         48        
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 50, 2, 16)         0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 50, 2, 32)         2080      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 50, 2, 32)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 50, 1, 64)         8256      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 50, 1, 64)         0         
_________________________________________________________________
dense_12 (Dense)             (None, 50, 1, 64)        

In [81]:
g_noise = tf.random.normal([1, 50, 2, 1])
x = np.random.randint(20, size=(1,50, 1))
x[:,1,0]

array([7])

In [106]:
y = [[0 for i in range(20)] for j in range(50)]
for i in range(50):
    for j in range(20):
        if x[:, i, 0] == j:
            y[i][j] = 3.5
        else:
            y[i][j] = 5.2

In [113]:
sparse_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [109]:
y = np.array(y)
y = y.reshape(1,50,20)

In [114]:
cross_entropy(0, 10)

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Exp}}; Op<name=Exp; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Exp]

# def symbol_classifier(blocksize):
    model = keras.Sequential()
    model.add(layers.Conv2D(16, (1, 2), padding='same', activation='relu', input_shape=(blocksize, 1, 2)))
    model.add(layers.AveragePooling2D((1, 1)))
    model.add(layers.Conv2D(32, (4,1), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D((1, 1)))
    model.add(layers.Conv2D(64, (4,1), padding='same', activation='relu'))
    model.add(layers.AveragePooling2D((1, 1)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(20))
    return model

In [9]:
test = symbol_classifier(50)
test.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 50, 1, 16)         80        
_________________________________________________________________
average_pooling2d (AveragePo (None, 50, 1, 16)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 50, 1, 32)         2080      
_________________________________________________________________
average_pooling2d_1 (Average (None, 50, 1, 32)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 50, 1, 64)         8256      
_________________________________________________________________
average_pooling2d_2 (Average (None, 50, 1, 64)         0         
_________________________________________________________________
dense_6 (Dense)              (None, 50, 1, 64)        

In [ ]:
# import math 
cons_4 = np.dot(np.sqrt(0.5),[complex(i,j)for i in c_4 for j in c_4])
cons_16 = np.array([complex(i,j)for j in c_16 for i in c_16r])
cons_16 = cons_16/np.sqrt(np.mean(np.abs(cons_16)**2))
print(cons_4,"\n",cons_16)

In [82]:
data1 = "hard"
data1_label = "hard_label"
data = dataset(data1, data1_label)
data

,real,imag,cons,block,label,buffer,label_real,label_imag
0,48.806980,-13.804771,2,1,15,0.948683-0.316228j,0.948683,-0.316228
1,-21.419588,-58.747407,2,1,18,0.316228-0.948683j,0.316228,-0.948683
2,61.033908,40.251547,2,1,7,0.948683+0.948683j,0.948683,0.948683
3,25.841522,72.678677,2,1,6,0.316228+0.948683j,0.316228,0.948683
4,40.576088,67.132076,2,1,7,0.948683+0.948683j,0.948683,0.948683
...,...,...,...,...,...,...,...,...
49995,28.789324,-41.060747,2,1000,18,0.316228-0.948683j,0.316228,-0.948683
49996,28.106421,-54.533358,2,1000,18,0.316228-0.948683j,0.316228,-0.948683
49997,-19.969952,-39.392252,2,1000,14,0.316228-0.316228j,0.316228,-0.316228
49998,-14.243322,5.572663,2,1000,13,-0.316228-0.316228j,-0.316228,-0.316228


In [6]:
def generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(20, use_bias=False, input_shape=[2,]))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(2))
    return model

In [7]:
generator_s = generator()

In [8]:
generator_n = generator()

In [9]:
generator_i = generator()

In [10]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(20, use_bias=False, input_shape=[2,]))
    model.add(layers.Dense(50, activation = 'sigmoid'))
    model.add(layers.Dense(1))
    return model

In [11]:
discriminator_t = make_discriminator_model()

In [12]:
discriminator_d = make_discriminator_model()

In [13]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [14]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [15]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [16]:
def identity_loss(real, fake):
    loss = tf.reduce_mean(tf.abs(real - fake))
    return LAMBDA * 0.5 * loss

In [22]:
generator_s_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_n_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_i_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_d_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_t_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [427]:
checkpoint_path = "./checkpoints/dis"

ckpt = tf.train.Checkpoint(generator_s=generator_s,
                           generator_n=generator_n,
                           generator_i=generator_i,
                           discriminator_t=discriminator_t,
                           discriminator_d=discriminator_d,
                           generator_s_optimizer=generator_s_optimizer,
                           generator_n_optimizer=generator_n_optimizer,
                           generator_i_optimizer=generator_i_optimizer,
                           discriminator_d_optimizer=discriminator_d_optimizer,
                           discriminator_t_optimizer=discriminator_t_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')